## Genarate Augmented Data 

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm

# Augmentation methods
def add_white_noise(data, noise_factor=0.005):
    noise = np.random.randn(len(data))
    augmented_data = data + noise_factor * noise
    return augmented_data.astype(type(data[0]))

def time_stretch(data, rate=1.0):
    return librosa.effects.time_stretch(data, rate=rate)

def time_shifting(data, shift_max=0.2):
    shift = np.random.randint(int(len(data) * shift_max))
    augmented_data = np.roll(data, shift)
    if shift > 0:
        augmented_data[:shift] = 0
    else:
        augmented_data[shift:] = 0
    return augmented_data

def pitch_shift(data, sample_rate, n_steps=2):
    return librosa.effects.pitch_shift(data, sr=sample_rate, n_steps=n_steps)

def random_gain(data, gain_min=0.8, gain_max=1.25):
    gain = np.random.uniform(gain_min, gain_max)
    return data * gain

# List of all augmentations to apply
augmentations = [
    lambda x: x,  # Original data
    add_white_noise,
    lambda x: time_stretch(x, rate=0.9),
    lambda x: time_stretch(x, rate=1.1),
    time_shifting,
    lambda x, sr: pitch_shift(x, sr, n_steps=2),
    lambda x, sr: pitch_shift(x, sr, n_steps=-2),
    random_gain
]

feature_normal = []
label_normal = []

# Load CSV file with metadata
df = pd.read_csv('./UrbanSound8K.csv')

# Loop over each file and apply augmentations
for index_num, row in tqdm(df.iterrows()):
    file_name = os.path.join(os.path.abspath('./data'), 'fold' + str(row["fold"]), str(row["slice_file_name"]))
    X, sample_rate = librosa.load(file_name)
    
    for augmentation in augmentations:
        if 'sr' in augmentation.__code__.co_varnames:  # Handle augmentations with sample rate argument
            augmented_X = augmentation(X, sample_rate)
        else:
            augmented_X = augmentation(X)
        mels = np.mean(librosa.feature.melspectrogram(y=augmented_X, sr=sample_rate).T, axis=0)
        feature_normal.append(mels)
        label_normal.append(row['classID'])
